# Install and load dependencies

In [ ]:
!pip install pycaret-ts-alpha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 468 kB 4.3 MB/s 
     |████████████████████████████████| 11.2 MB 53.3 MB/s 
     |████████████████████████████████| 3.3 MB 38.5 MB/s 
     |████████████████████████████████| 134 kB 67.7 MB/s 
     |████████████████████████████████| 79.9 MB 118 kB/s 
     |████████████████████████████████| 2.0 MB 43.7 MB/s 
     |████████████████████████████████| 303 kB 58.9 MB/s 
     |████████████████████████████████| 1.8 MB 22.5 MB/s 
     |████████████████████████████████| 69 kB 8.0 MB/s 
     |████████████████████████████████| 6.5 MB 24.4 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 1.6 MB 55.3 MB/s 
     |████████████████████████████████| 957 kB 62.1 MB/s 
     |████████████████████████████████| 34.5 MB 76.6 MB/s 
     |████████████████████████████████| 9.8 MB 40.3 MB/s 
  Created wheel for pyod: filename

# Load Data

In [ ]:
import pandas as pd
santa_rosa_df = pd.read_excel("/content/drive/MyDrive/Start Hackaton Datasets/Santa Rosa Monthly Water Consumption.xlsx")

In [ ]:
santa_rosa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   Date                                         60 non-null     datetime64[ns]
 1   Monthly Water Consumption (in cubic meters)  60 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.1 KB


In [ ]:
data = santa_rosa_df.groupby('Date').sum()

In [ ]:
data = data.asfreq('M')
data.head()

,Monthly Water Consumption (in cubic meters)
Date,
2016-01-31,159712800.0
2016-02-29,119784600.0
2016-03-31,179676900.0
2016-04-30,139748700.0
2016-05-31,159712800.0


# Pycaret

In [ ]:
from pycaret.time_series import TSForecastingExperiment
exp = TSForecastingExperiment()
exp.setup(data, fh = 12, fold = 3, session_id = 123)

INFO:logs:PyCaret TSForecastingExperiment
INFO:logs:Logging name: ts-default-name
INFO:logs:ML Usecase: MLUsecase.TIME_SERIES
INFO:logs:version 3.0.0
INFO:logs:Initializing setup()
INFO:logs:self.USI: f944
INFO:logs:self.variable_keys: {'variable_keys', 'fold_param', 'seasonality_present', 'primary_sp_to_use', '_ml_usecase', 'y', 'enforce_exogenous', 'fold_generator', '_all_metrics', 'n_jobs_param', '_all_models', 'index_type', 'USI', 'seed', 'fold_groups_param_full', 'X_train_transformed', 'X_transformed', 'all_sp_values', '_available_plots', '_gpu_n_jobs_param', 'enforce_pi', 'display_container', 'fh', 'y_test', 'html_param', 'memory', 'X_test', 'exp_id', 'X_test_transformed', 'log_plots_param', 'approach_type', '_all_models_internal', 'y_transformed', 'gpu_param', 'y_test_transformed', 'X_train', 'master_model_container', 'exp_name_log', 'y_train', 'logging_param', 'y_train_transformed', 'exogenous_present', 'data', 'idx', 'X', 'pipeline', 'experiment__', 'strictly_positive', 'seaso

,Description,Value
0,session_id,123
1,Target,Monthly Water Consumption (in cubic meters)
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Data shape,"(60, 1)"
5,Train data shape,"(48, 1)"
6,Test data shape,"(12, 1)"
7,Fold Generator,ExpandingWindowSplitter
8,Fold Number,3
9,Enforce Prediction Interval,False


INFO:logs:setup() successfully completed in 0.57s...............


In [ ]:
exp.plot_model(plot = 'ts')

INFO:logs:Visual Rendered Successfully


# Model Training

In [ ]:
best = exp.compare_models()

,Model,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2,TT (Sec)
snaive,Seasonal Naive Forecaster,25089416.6667,42873868.512,0.1253,0.1249,NaN,NaN,0.7898,0.02
grand_means,Grand Means Forecaster,60127908.9506,94104266.9203,0.3254,0.3037,NaN,NaN,-0.024,0.02
croston,Croston,61671587.5208,93226865.3067,0.35,0.3124,NaN,NaN,-0.006,0.02
polytrend,Polynomial Trend Forecaster,77159654.9874,99084814.8194,0.508,0.3885,NaN,NaN,-0.2254,0.03
naive,Naive Forecaster,149430361.1111,161212153.4573,1.0994,0.6414,NaN,NaN,-2.4977,1.25


INFO:logs:master_model_container: 47
INFO:logs:display_container: 2
INFO:logs:NaiveForecaster(sp=12)
INFO:logs:compare_models() successfully completed......................................


In [ ]:
# create fbprophet model
prophet = exp.create_model('prophet')
print(prophet)

,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,2016-12,7.260156e+07,8.302827e+07,0.5588,0.3868,NaN,NaN,-0.6770
1,2017-12,6.125174e+07,7.875828e+07,0.4066,0.3225,3.4086,2.4323,-0.0050
2,2018-12,4.391879e+07,8.268747e+07,0.2156,0.1921,2.5694,2.9843,0.6315
Mean,NaT,5.925736e+07,8.149134e+07,0.3937,0.3004,NaN,NaN,-0.0168
SD,NaT,1.179431e+07,1.937569e+06,0.1404,0.0810,NaN,NaN,0.5343


INFO:logs:master_model_container: 48
INFO:logs:display_container: 3
INFO:logs:ProphetPeriodPatched()
INFO:logs:create_model() successfully completed......................................


ProphetPeriodPatched()


In [ ]:
#Model Tuning
tuned_prophet = exp.tune_model(prophet)
print(tuned_prophet)

,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,2016-12,7.260248e+07,8.302927e+07,0.5588,0.3868,NaN,NaN,-0.6771
1,2017-12,6.125411e+07,7.875908e+07,0.4066,0.3225,3.4088,2.4323,-0.0050
2,2018-12,3.920461e+07,7.359266e+07,0.1960,0.1733,2.2936,2.6560,0.7081
Mean,NaT,5.768706e+07,7.846034e+07,0.3871,0.2942,NaN,NaN,0.0087
SD,NaT,1.386596e+07,3.858267e+06,0.1488,0.0894,NaN,NaN,0.5656


INFO:logs:master_model_container: 51
INFO:logs:display_container: 4
INFO:logs:ProphetPeriodPatched(changepoint_prior_scale=0.07052367291265604,
                     holidays_prior_scale=0.2772015819815348,
                     seasonality_mode='multiplicative',
                     seasonality_prior_scale=0.15960727025988383)
INFO:logs:tune_model() succesfully completed......................................


ProphetPeriodPatched(changepoint_prior_scale=0.07052367291265604,
                     holidays_prior_scale=0.2772015819815348,
                     seasonality_mode='multiplicative',
                     seasonality_prior_scale=0.15960727025988383)


# Deployment

In [ ]:
## Deployment
# finalize model
final_best = exp.finalize_model(best)# generate predictions
result = exp.predict_model(final_best, fh = 90)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(self=<pycaret.time_series.forecasting.oop.TSForecastingExperiment object at 0x7f2a701c06d0>, estimator=NaiveForecaster(sp=12), fit_kwargs=None, groups=None, model_only=True, experiment_custom_tags=None, display=None)
INFO:logs:Finalizing NaiveForecaster(sp=12)
INFO:logs:Initializing create_model()
INFO:logs:create_model(self=<pycaret.time_series.forecasting.oop.TSForecastingExperiment object at 0x7f2a701c06d0>, estimator=NaiveForecaster(sp=12), fold=None, round=4, cross_validation=True, predict=True, fit_kwargs={}, groups=None, refit=True, probability_threshold=None, experiment_custom_tags=None, verbose=False, system=False, add_to_model_list=False, metrics=None, display=None, kwargs={})
INFO:logs:Checking exceptions
INFO:logs:Importing libraries
INFO:logs:Copying training dataset
INFO:logs:Defining folds
INFO:logs:Declaring metric variables
INFO:logs:Importing untrained model
INFO:logs:Declaring custom model
INFO:logs:Sea

In [ ]:
# save the model
exp.save_model(final_best, 'santa_rosa_model')

INFO:logs:Initializing save_model()
INFO:logs:save_model(model=NaiveForecaster(sp=12), model_name=santa_rosa_model, prep_pipe_=PyCaretForecastingPipeline(steps=[('forecaster',
                                   TransformedTargetForecaster(steps=[('model',
                                                                       DummyForecaster())]))]), verbose=True, use_case=MLUsecase.TIME_SERIES, kwargs={})
INFO:logs:Adding model into prep_pipe
INFO:logs:santa_rosa_model.pkl saved in current working directory
INFO:logs:PyCaretForecastingPipeline(steps=[('forecaster',
                                   TransformedTargetForecaster(steps=[('model',
                                                                       NaiveForecaster(sp=12))]))])
INFO:logs:save_model() successfully completed......................................


Transformation Pipeline and Model Successfully Saved


(PyCaretForecastingPipeline(steps=[('forecaster',
                                    TransformedTargetForecaster(steps=[('model',
                                                                        NaiveForecaster(sp=12))]))]),
 'santa_rosa_model.pkl')

In [ ]:
import json
result = result["y_pred"].to_json(orient="table")
parsed = json.loads(result)

json_object = json.dumps(parsed, indent=4) 

In [ ]:
with open("predict_santa_rosa_map.json", "w") as outfile:
    json.dump(parsed, outfile)